### Chains with multiple inputs

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
import os
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain
from pydantic import SecretStr

llm = ChatOpenAI( 
  model=os.getenv("MODEL", ""),
  base_url=os.getenv("API_URL", ""),
  api_key=SecretStr(os.getenv("API_KEY", "")),
  temperature=0
)

prompt_template = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input={"input": "a parrot"})

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_3232\3518869110.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


{'input': 'a parrot',
 'text': 'Why did the parrot bring a ladder to the bar?\n\nBecause it heard the drinks were on the house! 🦜🍹'}

In [3]:
prompt_template = PromptTemplate(input_variables=["input", "language"], template="Tell me a joke about {input} in {language}. Don't think too much, just say the first joke that comes to mind.")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke({"input": "a parrot", "language": "Bengali"})

{'input': 'a parrot',
 'language': 'Bengali',
 'text': 'একটা পাখি (প্যারট) গাছের ডালে বসে ছিল।  \nঅচেনা লোক জিজ্ঞেস করল, “তুমি কী বলো?”  \n\nপ্যারট উত্তর দিল, “আমি তো ‘কী’ বলতে পারি না, আমি তো ‘কী-ওয়াই’!”'}

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [4]:
from langchain.chains.sequential import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {comment}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

# This is an LLMChain to translate a summary into German.
prompt_translation = PromptTemplate.from_template(
    template="Translate the summary to Bengali: \n\n {summary}"
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="bengali_translation"
)

In [6]:
overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name", "experience"],
    output_variables=["review", "comment", "summary", "bengali_translation"],
)

overall_chain.invoke({"dish_name": "Pizza Salami", "experience": "It was awful!"})

{'dish_name': 'Pizza Salami',
 'experience': 'It was awful!',
 'review': '**Title:** A Disappointing Slice of Disaster\n\nI’ve been a loyal fan of this pizzeria for years, but my recent order of the Pizza Salami has left me utterly disappointed—and I’m not just talking about the taste.\n\n**What Went Wrong**\n\n- **Toppings:** The salami was thin, dry, and oddly bland. It seemed like it had been sitting in the fridge for days before being placed on the pizza.\n- **Cheese:** Instead of a smooth, melty layer, the cheese was rubbery and stuck to the crust, making every bite feel more like chewing than eating.\n- **Crust:** The base was soggy in the middle and burnt at the edges. It didn’t have that satisfying crunch I’ve come to expect from their hand‑tossed dough.\n- **Overall Flavor:** There was a faint metallic aftertaste—likely from the overcooked salami—that clung to my palate long after the last bite.\n\n**Service & Delivery**\n\nThe pizza arrived 30 minutes late, and when it finall

Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [7]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

positive_template = """You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [8]:
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]

In [9]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
destination_chains

{'positive': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an AI that focuses on the positive side of things. Whenever you analyze a text, you look for the positive aspects and highlight them. Here is the text:\n{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001B8F7CBB4D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B8F8174050>, root_client=<openai.OpenAI object at 0x000001B8F7CB8EC0>, root_async_client=<openai.AsyncOpenAI object at 0x000001B8F7CBBCB0>, model_name='openai/gpt-oss-20b', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://127.0.0.1:1234/v1'), output_parser=StrOutputParser(), llm_kwargs={}),
 'neutral': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an AI

In [10]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

positive: Good for analyzing positive sentiments
neutral: Good for analyzing neutral sentiments
negative: Good for analyzing negative sentiments


In [11]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
print(router_prompt)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True,
)

chain.invoke({"input": "I ordered Pizza Salami for 9.99$ and it was awesome!"})

input_variables=['input'] input_types={} output_parser=RouterOutputParser() partial_variables={} template='Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_3232\1654321065.py:11: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


positive: {'input': 'I ordered Pizza Salami for 9.99$ and it was awesome!'}
> Finished chain.


{'input': 'I ordered Pizza Salami for 9.99$ and it was awesome!',
 'text': '**Positive Highlights**\n\n- **Great Value:** The pizza cost only $9.99—an affordable price for a tasty meal.\n- **Delicious Taste:** You described the pizza as “awesome,” indicating it met or exceeded your expectations in flavor and quality.\n- **Satisfaction:** Your overall experience was positive, showing that the order delivered enjoyment and satisfaction.\n\nIt sounds like you had a wonderful, budget‑friendly pizza night!'}